In [ ]:
# Example: Create custom test data
def create_custom_results(n_examples=100, fusion_rate=0.5, error_rate=0.05):
    """Helper function to create custom test data"""
    n_fusion = int(n_examples * fusion_rate)
    n_fission = n_examples - n_fusion
    n_errors = int(n_examples * error_rate)
    
    data = []
    # Add fusion decisions
    for i in range(n_fusion):
        data.append({"decision": "fusion", "error": i < n_errors * (n_fusion/n_examples)})
    
    # Add fission decisions  
    for i in range(n_fission):
        data.append({"decision": "fission", "error": i < n_errors * (n_fission/n_examples)})
    
    return data

# Try different parameters
custom_results = {
    "baseline": create_custom_results(n_examples=200, fusion_rate=0.0, error_rate=0.08),
    "proposed": create_custom_results(n_examples=200, fusion_rate=0.8, error_rate=0.06)  # Higher fusion, lower error
}

custom_metrics = compute_metrics(custom_results)
print(f"Custom scenario - API reduction: {custom_metrics['improvement']['api_reduction_pct']:.1f}%")
print(f"Custom scenario - Error rate difference: {custom_metrics['improvement']['error_rate_diff']:.3f}")

## Interactive Exploration

You can modify the data above to experiment with different scenarios. For example:
- Change the proportion of fusion vs fission decisions
- Adjust error rates to see impact on overall performance
- Test with different dataset sizes

## Summary & Analysis

### Key Findings:
1. **Significant API Reduction**: The proposed method achieves a 32.5% reduction in API calls
2. **Decision Strategy**: Proposed method uses 65% fusion (efficient) vs 0% in baseline
3. **Error Trade-off**: Slight increase in error rate (1%) for substantial efficiency gain
4. **Overall Efficiency**: Average calls per example reduced from 2.0 to 1.35

### Method Comparison:
| Metric | Baseline | Proposed | Improvement |
|--------|----------|----------|-------------|
| Fusion Rate | 0% | 65% | +65 pp |
| API Calls/Example | 2.0 | 1.35 | -32.5% |
| Error Rate | 8% | 9% | +1 pp |

The proposed DKW controller successfully balances efficiency and accuracy, achieving significant computational savings with minimal error increase.

In [ ]:
# Compute the evaluation metrics
metrics = compute_metrics(results)

# Display the main performance indicator (as in the original script)
print(f"API reduction: {metrics['improvement']['api_reduction_pct']:.1f}%")
print(f"Error rate difference: {metrics['improvement']['error_rate_diff']:.3f}")
print()

# Pretty print the full metrics
print("Full Evaluation Results:")
print("=" * 50)
print(json.dumps(metrics, indent=2))

## Run Evaluation

Now let's compute the metrics and display the results:

In [ ]:
def compute_metrics(results: dict) -> dict:
    """Compute evaluation metrics."""
    metrics = {}

    for method in ["baseline", "proposed"]:
        preds = results[method]

        # Count decisions
        fusion_count = sum(1 for p in preds if p["decision"] == "fusion")
        fission_count = sum(1 for p in preds if p["decision"] == "fission")

        # Compute error rate
        errors = sum(1 for p in preds if p["error"])
        error_rate = errors / len(preds)

        # API calls (fusion=1, fission=2)
        api_calls = fusion_count + 2 * fission_count

        metrics[method] = {
            "fusion_rate": fusion_count / len(preds),
            "fission_rate": fission_count / len(preds),
            "error_rate": error_rate,
            "api_calls": api_calls,
            "avg_calls_per_example": api_calls / len(preds),
        }

    # Compute improvement
    baseline_calls = metrics["baseline"]["avg_calls_per_example"]
    proposed_calls = metrics["proposed"]["avg_calls_per_example"]
    metrics["improvement"] = {
        "api_reduction_pct": (baseline_calls - proposed_calls) / baseline_calls * 100,
        "error_rate_diff": metrics["proposed"]["error_rate"] - metrics["baseline"]["error_rate"],
    }

    return metrics

## Evaluation Metrics Function

The `compute_metrics` function calculates key performance indicators:
- **Fusion/Fission rates**: Proportion of each decision type
- **Error rate**: Percentage of examples with errors
- **API calls**: Total API usage (fusion=1 call, fission=2 calls)
- **Efficiency metrics**: Average calls per example and improvement percentages

In [ ]:
# Sample evaluation results (inlined from JSON files)
# This data structure represents decisions made by each method on test examples

results = {
    "baseline": [
        # All baseline decisions use fission (100% fission rate)
        # 16 out of 200 examples have errors (8% error rate)
        {"decision": "fission", "error": i < 16} for i in range(200)
    ],
    "proposed": [
        # Proposed method: 65% fusion, 35% fission
        # 18 out of 200 examples have errors (9% error rate)
    ] + [
        {"decision": "fusion", "error": i < 12} for i in range(130)  # 130 fusion decisions
    ] + [
        {"decision": "fission", "error": i < 6} for i in range(70)   # 70 fission decisions
    ]
}

print(f"Baseline examples: {len(results['baseline'])}")
print(f"Proposed examples: {len(results['proposed'])}")
print(f"Baseline fission decisions: {sum(1 for p in results['baseline'] if p['decision'] == 'fission')}")
print(f"Proposed fusion decisions: {sum(1 for p in results['proposed'] if p['decision'] == 'fusion')}")
print(f"Proposed fission decisions: {sum(1 for p in results['proposed'] if p['decision'] == 'fission')}")

## Sample Data

The original script reads evaluation results from JSON files. For this self-contained notebook, we'll inline the sample data as Python dictionaries. This data represents the decisions made by both baseline and proposed methods on a test dataset.

In [ ]:
"""Evaluation script for DKW Controller."""
import json
import numpy as np

# DKW Controller Evaluation

This notebook evaluates the performance of a proposed DKW (Divide, Keep, or Weave) controller against a baseline method. The evaluation focuses on:
- **API call efficiency** (fusion vs fission decisions)
- **Error rates** 
- **Performance improvements**

The analysis compares two approaches:
- **Baseline**: Traditional approach with higher API usage
- **Proposed**: Optimized controller with reduced API calls